In [ ]:
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline
from quantopian.pipeline.filters.morningstar import Q1500US
from quantopian.pipeline.data.sentdex import sentiment
from quantopian.pipeline.data.morningstar import operation_ratios

def make_pipeline():
    # Yes: operation_ratios.revenue_growth, operation_ratios.operation_margin, sentiment

    testing_factor1 = operation_ratios.operation_margin.latest
    testing_factor2 = operation_ratios.revenue_growth.latest
    testing_factor3 = sentiment.sentiment_signal.latest

    universe = (Q1500US() &
               testing_factor1.notnull() &
               testing_factor2.notnull() &
               testing_factor3.notnull())

    testing_factor1 = testing_factor1.rank(mask=universe, method='average')
    testing_factor2 = testing_factor2.rank(mask=universe, method='average')
    testing_factor3 = testing_factor3.rank(mask=universe, method='average')

    testing_factor = testing_factor1 + testing_factor2 + testing_factor3

    testing_quantiles = testing_factor.quantiles(2)

    pipe = Pipeline(columns={
            'testing_factor':testing_factor,
            'shorts':testing_quantiles.eq(0),
            'longs':testing_quantiles.eq(1)},

                   screen=universe)
    return pipe

result = run_pipeline(make_pipeline(), start_date='2015-01-01', end_date='2016-01-01')
result.head()

In [1]:
assets = result.index.levels[1].unique()
pricing = get_pricing(assets, start_date='2014-12-01', end_date='2016-02-01', fields='open_price')
len(assets)

NameError: name 'result' is not defined

In [ ]:
import alphalens

alphalens.tears.create_factor_tear_sheet(factor = result['testing_factor'],
                                        prices = pricing,
                                        quantiles = 2,
                                        periods = (3, 10, 30))